### 1) Merge processed community data with original data to get original tag data.

In [25]:
import pickle
with open(file = 'post_cluster_pre.pickle', mode = 'rb') as file:
    data = pickle.load(file)

In [26]:
data

,id,creation_date,body,tags,community
0,69006423,2021-09-01 00:00:35.237,<p>Using interceptors for the first time to re...,"[nodeDOTjs, reactjs, express, axios, refreshto...",0
1,69006426,2021-09-01 00:00:55.583,"<p>The APP below uses <code>navbarPage</code>,...","[r, shiny]",17
2,69006431,2021-09-01 00:01:23.670,<p>I am developing a java project and every th...,"[java, installation, compilation, project, ope...",3
3,69006437,2021-09-01 00:03:18.750,"<p>As you know, all firebase hosting is provid...","[firebase, nuxtDOTjs, firebasehosting]",5
4,69006439,2021-09-01 00:03:36.930,<p>I am trying to split the string as below</p...,"[r, dataDOTtable]",17
...,...,...,...,...,...
2713807,77019848,2023-08-31 23:54:30.057,<p>I need help solving a CORS issue.</p>\n<p>I...,"[docker, dockercompose, owaspdependencycheck, ...",7
2713808,77019849,2023-08-31 23:55:21.660,<p>I am currently attempting to search a Share...,"[microsoftgraphapi, sharepointonline]",1
2713809,77019852,2023-08-31 23:56:42.653,"<p>Similar to <a href=""https://stackoverflow.c...","[python, numpy, floatingpoint]",17
2713810,77019854,2023-08-31 23:57:28.633,<p>I utilized the guidelines presented in <a h...,"[angular, typescript, progressivewebapps, ngsw...",0


In [36]:
# Import Modules
import pandas as pd
import numpy as np
import sqlite3
import pandas as pd
from nltk import FreqDist
import pickle
import math
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm

# Import Dataset
conn = sqlite3.connect('/data1/StackOverflow/stackexchange-to-sqlite/stack.db')
query = '''
SELECT id, creation_date, tags
FROM questions
WHERE creation_date > '2021-09-01'
AND creation_date < '2023-09-01';
'''
df_tags = pd.read_sql_query(query, conn)
conn.close()

In [37]:
df_tags

,id,creation_date,tags
0,69006423,2021-09-01 00:00:35.237,"[""node.js"", ""reactjs"", ""express"", ""axios"", ""re..."
1,69006426,2021-09-01 00:00:55.583,"[""r"", ""shiny""]"
2,69006431,2021-09-01 00:01:23.670,"[""java"", ""installation"", ""compilation"", ""proje..."
3,69006437,2021-09-01 00:03:18.750,"[""firebase"", ""nuxt.js"", ""firebase-hosting""]"
4,69006439,2021-09-01 00:03:36.930,"[""r"", ""data.table""]"
...,...,...,...
2715204,77019848,2023-08-31 23:54:30.057,"[""docker"", ""docker-compose"", ""owasp-dependency..."
2715205,77019849,2023-08-31 23:55:21.660,"[""microsoft-graph-api"", ""sharepoint-online""]"
2715206,77019852,2023-08-31 23:56:42.653,"[""python"", ""numpy"", ""floating-point""]"
2715207,77019854,2023-08-31 23:57:28.633,"[""angular"", ""typescript"", ""progressive-web-app..."


In [40]:
df_tags = pd.merge(data, df_tags, on = 'id')[['id', 'creation_date_x', 'body', 'tags_y', 'community']]
df_tags.columns = ['id', 'creation_date', 'body', 'tags', 'community']
df_tags

,id,creation_date,body,tags,community
0,69006423,2021-09-01 00:00:35.237,<p>Using interceptors for the first time to re...,"[""node.js"", ""reactjs"", ""express"", ""axios"", ""re...",0
1,69006426,2021-09-01 00:00:55.583,"<p>The APP below uses <code>navbarPage</code>,...","[""r"", ""shiny""]",17
2,69006431,2021-09-01 00:01:23.670,<p>I am developing a java project and every th...,"[""java"", ""installation"", ""compilation"", ""proje...",3
3,69006437,2021-09-01 00:03:18.750,"<p>As you know, all firebase hosting is provid...","[""firebase"", ""nuxt.js"", ""firebase-hosting""]",5
4,69006439,2021-09-01 00:03:36.930,<p>I am trying to split the string as below</p...,"[""r"", ""data.table""]",17
...,...,...,...,...,...
2713807,77019848,2023-08-31 23:54:30.057,<p>I need help solving a CORS issue.</p>\n<p>I...,"[""docker"", ""docker-compose"", ""owasp-dependency...",7
2713808,77019849,2023-08-31 23:55:21.660,<p>I am currently attempting to search a Share...,"[""microsoft-graph-api"", ""sharepoint-online""]",1
2713809,77019852,2023-08-31 23:56:42.653,"<p>Similar to <a href=""https://stackoverflow.c...","[""python"", ""numpy"", ""floating-point""]",17
2713810,77019854,2023-08-31 23:57:28.633,<p>I utilized the guidelines presented in <a h...,"[""angular"", ""typescript"", ""progressive-web-app...",0


### 1) Preprocessing

In [41]:
import pandas as pd
from nltk import FreqDist
def wc(text):
    """
    Cleaning function to be used with our first wordcloud
    """
    
    if text:
        tags = text.replace('><',' ')
        tags = tags.replace('-','')
        tags = tags.replace('.','DOT')
        tags = tags.replace('c++','Cpp')
        tags = tags.replace('c#','Csharp')
        tags = tags.replace('>','')
        return tags.replace('<','')
    else:
        return 'None'
    
def clean_tags(text):
    """
    Cleaning function for tags
    """
    
    if text:
        tags = text.replace('><',' ')
        tags = tags.replace('>','')
        return tags.replace('<','')
    else:
        return 'None'
    
def tag_freq(data):
    tags = data['tags'].str.replace('[\["\]]', '', regex=True)
    tags = [tag for i in tags.apply(lambda x: wc(x)) for tag in i.split(', ')]
    result = FreqDist(tags)
    return result
# Preprocessing
df_tags['creation_date'] = pd.to_datetime(df_tags['creation_date'])
df_tags['year_month'] = df_tags['creation_date'].dt.to_period('D')
df_tags['year_month'] = df_tags['year_month'].astype(str)
year_month = df_tags.year_month.unique()

In [42]:
df_tags

,id,creation_date,body,tags,community,year_month
0,69006423,2021-09-01 00:00:35.237,<p>Using interceptors for the first time to re...,"[""node.js"", ""reactjs"", ""express"", ""axios"", ""re...",0,2021-09-01
1,69006426,2021-09-01 00:00:55.583,"<p>The APP below uses <code>navbarPage</code>,...","[""r"", ""shiny""]",17,2021-09-01
2,69006431,2021-09-01 00:01:23.670,<p>I am developing a java project and every th...,"[""java"", ""installation"", ""compilation"", ""proje...",3,2021-09-01
3,69006437,2021-09-01 00:03:18.750,"<p>As you know, all firebase hosting is provid...","[""firebase"", ""nuxt.js"", ""firebase-hosting""]",5,2021-09-01
4,69006439,2021-09-01 00:03:36.930,<p>I am trying to split the string as below</p...,"[""r"", ""data.table""]",17,2021-09-01
...,...,...,...,...,...,...
2713807,77019848,2023-08-31 23:54:30.057,<p>I need help solving a CORS issue.</p>\n<p>I...,"[""docker"", ""docker-compose"", ""owasp-dependency...",7,2023-08-31
2713808,77019849,2023-08-31 23:55:21.660,<p>I am currently attempting to search a Share...,"[""microsoft-graph-api"", ""sharepoint-online""]",1,2023-08-31
2713809,77019852,2023-08-31 23:56:42.653,"<p>Similar to <a href=""https://stackoverflow.c...","[""python"", ""numpy"", ""floating-point""]",17,2023-08-31
2713810,77019854,2023-08-31 23:57:28.633,<p>I utilized the guidelines presented in <a h...,"[""angular"", ""typescript"", ""progressive-web-app...",0,2023-08-31


In [43]:
all_keys = tag_freq(df_tags)
all_keys = pd.DataFrame(all_keys, index = ['tag']).transpose().reset_index()
all_keys

,index,tag
0,nodeDOTjs,82196
1,reactjs,155462
2,express,18287
3,axios,8260
4,refreshtoken,369
...,...,...
49135,taffyDOTjs,1
49136,swintransformer,1
49137,httpupload,1
49138,microsoftDOTwebDOTadministration,1


In [45]:
num_community = df_tags.community.unique()
num_community.sort()

In [ ]:
# Extract keys througout the whole data
all_keys = tag_freq(df_tags)
all_keys = pd.DataFrame(all_keys, index = ['tag']).transpose().reset_index()

for i in range(len(year_month)):
    data = df_tags[df_tags['year_month'] == year_month[i]]
    for j in range(len(num_community)):
        data_community = data[data['community']==j]
        tags = tag_freq(data_community)
        tagCount = pd.DataFrame(tags, index = ['tag']).transpose().reset_index()
        tagShare = []
        for k in range(len(tagCount)):
            tagShare.append((tagCount['tag'][k] / tagCount['tag'].sum())*100)
        tagCount['tagShare'] = tagShare
        varName = year_month[i].replace('-', '_')+f'_{j}'
        tagCount = tagCount.rename(columns = {'tag':f'tag_{varName}','tagShare':f'tagShare_{varName}'})
        all_keys = pd.merge(all_keys, tagCount, on = 'index', how = 'left')

In [ ]:
# Save data
#with open(file = 'cluster_tag_share.pickle', mode = 'wb') as file:
#    pickle.dump(all_keys, file)

### 3) Calculate Entropy for each day and each cluster

In [49]:
import pickle
with open(file = 'cluster_tag_share.pickle', mode = 'rb') as file:
    all_keys = pickle.load(file)

In [50]:
all_keys

,index,tag,tag_2021_09_01_0,tagShare_2021_09_01_0,tag_2021_09_01_1,tagShare_2021_09_01_1,tag_2021_09_01_2,tagShare_2021_09_01_2,tag_2021_09_01_3,tagShare_2021_09_01_3,...,tag_2023_08_31_13,tagShare_2023_08_31_13,tag_2023_08_31_14,tagShare_2023_08_31_14,tag_2023_08_31_15,tagShare_2023_08_31_15,tag_2023_08_31_16,tagShare_2023_08_31_16,tag_2023_08_31_17,tagShare_2023_08_31_17
0,nodeDOTjs,82196,162.0,4.295943,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,reactjs,155462,271.0,7.186423,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.044405
2,express,18287,36.0,0.954654,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,axios,8260,14.0,0.371254,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,refreshtoken,369,1.0,0.026518,2.0,0.164745,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49135,taffyDOTjs,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49136,swintransformer,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.044405
49137,httpupload,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49138,microsoftDOTwebDOTadministration,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 4) Calculate Entropy

In [51]:
all_keys.isna().all()
# 아예 모두가 NA value이면 -> Entropy = 0
# 극 소수의 non-NA value가 있으면 -> Entropy = negative value.

index                     False
tag                       False
tag_2021_09_01_0          False
tagShare_2021_09_01_0     False
tag_2021_09_01_1          False
                          ...  
tagShare_2023_08_31_15    False
tag_2023_08_31_16          True
tagShare_2023_08_31_16     True
tag_2023_08_31_17         False
tagShare_2023_08_31_17    False
Length: 26282, dtype: bool

In [52]:
import math
import numpy as np

def calculate_entropy(probabilities):
    """ Calculate the Shannon entropy of a given list of probabilities. """
    entropy = 0
    for p in probabilities:
        if p > 0:
            entropy += p * math.log(p, 2)
    return -entropy

In [53]:
# Measure score
entropy_Score = []
# Calculate HHI for each monthly tag share column.
for i in range(3, all_keys.shape[1], 2):
    arr = np.array(all_keys.iloc[:, i])
    arr = arr/100
    arrList = arr.tolist()
    entropy_Score.append(calculate_entropy(arrList))

In [54]:
len(entropy_Score)

13140

In [56]:
# Save data
#with open(file = 'ent_Score2.pickle', mode = 'wb') as file:
#    pickle.dump(entropy_Score, file)